In [37]:
import sqlite3
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
#import streamlit as st
import time
import matplotlib.pyplot as plt


In [2]:
pip install sqlite3

^C
Note: you may need to restart the kernel to use updated packages.


In [38]:


connexion = sqlite3.connect("imdb.db")
cursor = connexion.cursor()



In [44]:


# creating dataframe for movie details table
cursor.execute("""
    SELECT tb.primaryTitle, tb.startYear, tb.genres, tc.directors, tr.averageRating, tr.numVotes
    FROM title_basics tb, title_crew tc, title_ratings tr 
    ON tb.tconst = tc.tconst AND tb.tconst = tr.tconst
    where tb.isAdult = 0
""")
data = cursor.fetchall()
col = [description[0] for description in cursor.description]

df = pd.DataFrame.from_records(data=data, columns=col)



In [45]:
df=df.drop_duplicates()

df.shape



(994051, 6)

In [46]:
df.head()

,primaryTitle,startYear,genres,directors,averageRating,numVotes
0,Carmencita,1894,"Documentary,Short",nm0005690,5.7,1924
2,Le clown et ses chiens,1892,"Animation,Short",nm0721526,5.8,259
4,Pauvre Pierrot,1892,"Animation,Comedy,Romance",nm0721526,6.5,1737
6,Un bon bock,1892,"Animation,Short",nm0721526,5.6,174
8,Blacksmith Scene,1893,"Comedy,Short",nm0005690,6.2,2550


In [47]:


df['averageRating'] = df['averageRating'].astype(float)
df['numVotes'] = df['numVotes'].astype(int)



In [48]:


df_above5 = df[df['averageRating'] >= 8.0]
df_above5



df_clean = df_above5[df_above5['numVotes'] >= 1000]
df_clean



,primaryTitle,startYear,genres,directors,averageRating,numVotes
578,A Trip to the Moon,1902,"Action,Adventure,Comedy",nm0617588,8.2,50411
5344,The Cabinet of Dr. Caligari,1920,"Horror,Mystery,Thriller",nm0927468,8.0,64645
5940,One Week,1920,"Comedy,Short","nm0166836,nm0000036",8.1,8274
6358,The Kid,1921,"Comedy,Drama,Family",nm0000122,8.3,126941
6368,The Phantom Carriage,1921,"Drama,Fantasy,Horror",nm0803705,8.0,12789
...,...,...,...,...,...,...
1987638,Vaulter,2019,Drama,nm0661238,8.5,3581
1987640,Hunting,2019,Drama,nm0661238,9.0,4262
1987700,The Winter Line,2020,"Drama,Mystery,Sci-Fi",nm0003557,8.2,9150
1987810,Kaithi,2019,"Action,Adventure,Crime",nm7992231,8.5,32445


In [49]:
df_clean['genres'] = df_clean['genres'].astype("string")
df_clean['directors'] = df_clean['directors'].astype("string")

/tmp/ipykernel_11493/4016758238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['genres'] = df_clean['genres'].astype("string")
/tmp/ipykernel_11493/4016758238.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['directors'] = df_clean['directors'].astype("string")


In [50]:


df_clean.dtypes



primaryTitle      object
startYear         object
genres            string
directors         string
averageRating    float64
numVotes           int64
dtype: object

In [51]:
df_clean.head()

,primaryTitle,startYear,genres,directors,averageRating,numVotes
578,A Trip to the Moon,1902,"Action,Adventure,Comedy",nm0617588,8.2,50411
5344,The Cabinet of Dr. Caligari,1920,"Horror,Mystery,Thriller",nm0927468,8.0,64645
5940,One Week,1920,"Comedy,Short","nm0166836,nm0000036",8.1,8274
6358,The Kid,1921,"Comedy,Drama,Family",nm0000122,8.3,126941
6368,The Phantom Carriage,1921,"Drama,Fantasy,Horror",nm0803705,8.0,12789


In [52]:


# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''



In [53]:


# Apply clean_data function to your features.
features = ['directors', 'genres']

for feature in features:
    df_clean[feature] = df_clean[feature].apply(clean_data)



/tmp/ipykernel_11493/2218832250.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[feature] = df_clean[feature].apply(clean_data)


In [54]:


def create_soup(x):
    return ' '.join(x['directors']) + ' ,' + ' '.join(x['genres'])



In [55]:


# Create a new soup feature
df_clean['soup'] = df_clean.apply(create_soup, axis=1)



/tmp/ipykernel_11493/35972400.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['soup'] = df_clean.apply(create_soup, axis=1)


In [56]:


df_clean['soup'].head()
print(df_clean['soup'][0])



KeyError: 0

In [57]:
features = ['soup']

for feature in features:
    df_clean[feature] = df_clean[feature].apply(clean_data)

/tmp/ipykernel_11493/55744466.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[feature] = df_clean[feature].apply(clean_data)


In [58]:
print(df_clean['soup'])

578                        nm0617588,action,adventure,comedy
5344                       nm0927468,horror,mystery,thriller
5940                        nm0166836,nm0000036,comedy,short
6358                           nm0000122,comedy,drama,family
6368                          nm0803705,drama,fantasy,horror
                                 ...                        
1987638                                      nm0661238,drama
1987640                                      nm0661238,drama
1987700                       nm0003557,drama,mystery,sci-fi
1987810                     nm7992231,action,adventure,crime
1988106    nm2967345,nm2013928,nm1267452,action,adventure...
Name: soup, Length: 14891, dtype: object


In [59]:
# Function to convert all strings to lower case and strip names of spaces
df_clean['soup'] = df_clean['soup'].replace(',', ' ', regex=True)

/tmp/ipykernel_11493/2337589423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['soup'] = df_clean['soup'].replace(',', ' ', regex=True)


In [60]:


df_clean.head()



,primaryTitle,startYear,genres,directors,averageRating,numVotes,soup
578,A Trip to the Moon,1902,"action,adventure,comedy",nm0617588,8.2,50411,nm0617588 action adventure comedy
5344,The Cabinet of Dr. Caligari,1920,"horror,mystery,thriller",nm0927468,8.0,64645,nm0927468 horror mystery thriller
5940,One Week,1920,"comedy,short","nm0166836,nm0000036",8.1,8274,nm0166836 nm0000036 comedy short
6358,The Kid,1921,"comedy,drama,family",nm0000122,8.3,126941,nm0000122 comedy drama family
6368,The Phantom Carriage,1921,"drama,fantasy,horror",nm0803705,8.0,12789,nm0803705 drama fantasy horror


In [61]:


df_clean['primaryTitle'] = df_clean['primaryTitle'].astype("string")



/tmp/ipykernel_11493/2863556602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['primaryTitle'] = df_clean['primaryTitle'].astype("string")


In [62]:


df_clean = df_clean.reset_index()



In [118]:
df_clean.head(300)

,index,primaryTitle,startYear,genres,directors,averageRating,numVotes,soup
0,578,A Trip to the Moon,1902,"action,adventure,comedy",nm0617588,8.2,50411,nm0617588 action adventure comedy
1,5344,The Cabinet of Dr. Caligari,1920,"horror,mystery,thriller",nm0927468,8.0,64645,nm0927468 horror mystery thriller
2,5940,One Week,1920,"comedy,short","nm0166836,nm0000036",8.1,8274,nm0166836 nm0000036 comedy short
3,6358,The Kid,1921,"comedy,drama,family",nm0000122,8.3,126941,nm0000122 comedy drama family
4,6368,The Phantom Carriage,1921,"drama,fantasy,horror",nm0803705,8.0,12789,nm0803705 drama fantasy horror
...,...,...,...,...,...,...,...,...
295,64670,The Emperor's Naked Army Marches On,1987,"documentary,war",nm0361672,8.2,1912,nm0361672 documentary war
296,64824,Full Metal Jacket,1987,"drama,war",nm0000040,8.3,741465,nm0000040 drama war
297,64942,Hachi-ko,1987,"biography,drama,family",nm0477771,8.0,5350,nm0477771 biography drama family
298,65048,Wings of Desire,1987,"drama,fantasy,romance",nm0000694,8.0,71681,nm0000694 drama fantasy romance


In [114]:
df_clean[df_clean['primaryTitle']=='Titanic']

,index,primaryTitle,startYear,genres,directors,averageRating,numVotes,soup


In [64]:


# Import CountVectorizer and create the count matrix
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df_clean['soup'])



In [73]:


count_matrix.shape
print(count_matrix)



  (0, 0)	1
  (0, 1)	1
  (0, 4)	1
  (0, 3149)	1
  (1, 14)	1
  (1, 17)	1
  (1, 4483)	1
  (1, 10040)	1
  (2, 4)	1
  (2, 21)	1
  (2, 1118)	1
  (2, 10037)	1
  (3, 4)	1
  (3, 7)	1
  (3, 8)	1
  (3, 31)	1
  (4, 7)	1
  (4, 9)	1
  (4, 14)	1
  (4, 3925)	1
  (5, 0)	1
  (5, 4)	1
  (5, 3190)	1
  (5, 4157)	1
  (5, 10040)	1
  :	:
  (14884, 4)	1
  (14884, 8236)	1
  (14885, 0)	1
  (14885, 1)	1
  (14885, 2)	1
  (14885, 7310)	1
  (14886, 7)	1
  (14886, 3327)	1
  (14887, 7)	1
  (14887, 3327)	1
  (14888, 7)	1
  (14888, 10)	1
  (14888, 17)	1
  (14888, 308)	1
  (14888, 10036)	1
  (14889, 0)	1
  (14889, 1)	1
  (14889, 5)	1
  (14889, 9744)	1
  (14890, 0)	1
  (14890, 1)	1
  (14890, 2)	1
  (14890, 5537)	1
  (14890, 6968)	1
  (14890, 8103)	1


In [66]:
# Compute the Cosine Similarity matrix based on the count_matrix
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [76]:


print(cosine_sim2.shape)
print(cosine_sim2.dtypes)

print(cosine_sim2)



(14891, 14891)


AttributeError: 'numpy.ndarray' object has no attribute 'dtypes'

In [109]:


# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim2):
    # Get the index of the movie that matches the title
    idx = df_clean.index[df_clean['primaryTitle'] == title]
    print(f"1. idx : {idx}")
    print(idx)

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = cosine_sim2[idx]
    print(f"2. sim_scores : {sim_scores}")
    print(sim_scores.shape)
    df_sim=pd.DataFrame(sim_scores.reshape(-1),columns=['cos'])

    df_sim.reset_index()

    df_sim.columns
    # print(1000000000000000000)
    # print(df_sim.head())

    rslt_df = df_sim.sort_values(by='cos',ascending=False)

    # print(rslt_df[0:11])
    rec_list=rslt_df.reset_index()
    list1=[]
    list1=rec_list['index'][1:11].values

    # Return the top 10 most similar movies
    return df_clean['primaryTitle'].iloc[list1]


In [139]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations_list(liked_movies_list, cosine_sim=cosine_sim2):
    # Get the index of the movie that matches the title
    
    idx_list=[]
    for m in liked_movies_list:
        
        idx = df_clean.index[df_clean['primaryTitle'] == m]
        idx_list.append(idx)

    df_sim_list=pd.DataFrame()
    print(idx_list)
    for m in idx_list:
        sim_scores = cosine_sim2[m]
        df_sim=pd.DataFrame(sim_scores.reshape(-1),columns=[m])
        df_sim_list[m]=df_sim



        
    df_sim_list['average']=df_sim_list.mean(numeric_only=True, axis=1)

    rslt_df = df_sim_list.sort_values(by='average',ascending=False)

    # print(rslt_df[0:11])
    rec_list=rslt_df.reset_index()
    list1=[]
    list1=rec_list['index'][0:11].values

    # Return the top 10 most similar movies
    return df_clean['primaryTitle'].iloc[list1]

In [140]:
liked_movies_list=['One Week','Full Metal Jacket','The Kid']

var = get_recommendations_list(liked_movies_list)
print(var)

[Int64Index([2], dtype='int64'), Int64Index([296], dtype='int64'), Int64Index([3], dtype='int64')]
40                The Great Dictator
3                            The Kid
16                     The Cameraman
868      Goodbye, Farewell, and Amen
966           Devils on the Doorstep
3740                Abyssinia, Henry
296                Full Metal Jacket
1131                   Laughing Eyes
12073                     Reply 1988
9469         My Grandfather's People
9882                 #Kaadal Kahaani
Name: primaryTitle, dtype: string


In [131]:
liked_movies_list=['One Week','Full Metal Jacket','The Kid']
idx_list=[]
for m in liked_movies_list:
    
    idx = df_clean.index[df_clean['primaryTitle'] == m]
    idx_list.append(idx)

df_sim_list=pd.DataFrame()
print(idx_list)
for m in idx_list:
    sim_scores = cosine_sim2[m]
    df_sim=pd.DataFrame(sim_scores.reshape(-1),columns=[m])
    df_sim_list[m]=df_sim



    
df_sim_list['average']=df_sim_list.mean(numeric_only=True, axis=1)

rslt_df = df_sim_list.sort_values(by='average',ascending=False)

# print(rslt_df[0:11])
rec_list=rslt_df.reset_index()
list1=[]
list1=rec_list['index'][1:11].values

print(list1)

[Int64Index([2], dtype='int64'), Int64Index([296], dtype='int64'), Int64Index([3], dtype='int64')]
[    3    16   868   966  3740   296  1131 12073  9469  9882]


In [108]:


idx = df_clean.index[df_clean['primaryTitle'] == 'One Week']
print(f"1. idx : {idx}")
print(idx)

# Get the pairwsie similarity scores of all movies with that movie
sim_scores = cosine_sim2[idx]
print(f"2. sim_scores : {sim_scores}")
print(sim_scores.shape)
df_sim=pd.DataFrame(sim_scores.reshape(-1),columns=['cos'])

df_sim.reset_index()

df_sim.columns
# print(1000000000000000000)
# print(df_sim.head())

rslt_df = df_sim.sort_values(by='cos',ascending=False)

# print(rslt_df[0:11])
rec_list=rslt_df.reset_index()
list1=[]
list1=rec_list['index'][1:11].values
df_clean['primaryTitle'].iloc[list1]

1. idx : Int64Index([2], dtype='int64')
Int64Index([2], dtype='int64')
2. sim_scores : [[0.25 0.   1.   ... 0.   0.   0.  ]]
(1, 14891)


14792                                     SodaPop
12304    The Gay and Wondrous Life of Caleb Gallo
41                          A Plumbing We Will Go
12693                                  Human Cake
9169                                 The Bus Stop
1165                                   The Parlor
14340                          Leading Lady Parts
5809               A Parks and Recreation Special
1037                      Casper & Mandrilaftalen
4299                 24 Season 5: Logan's Retreat
Name: primaryTitle, dtype: string

In [119]:


var = get_recommendations('Full Metal Jacket')
print(var)



1. idx : Int64Index([296], dtype='int64')
Int64Index([296], dtype='int64')
2. sim_scores : [[0.         0.         0.         ... 0.25819889 0.         0.        ]]
(1, 14891)
8917             Richard II
108         The Deer Hunter
282                 Platoon
159                The Boat
4916      Dara of Jasenovac
33       The Grand Illusion
150             The Shining
294       Goodbye, Children
14778    Tuntematon sotilas
648               Flashback
Name: primaryTitle, dtype: string
